### Задача

https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/

In [1]:
import numpy as np
import pandas as pd
import scipy
import re

import xgboost as xgb
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score

import multiprocessing
import datetime
from pycm import *

In [2]:
from nltk.corpus import stopwords
from collections import Counter

russian_stopwords = stopwords.words("russian")
stopwords_rus_dict = Counter(russian_stopwords)

In [3]:
#фиксация random_state
SEED = 42

In [4]:
#выгрузка всех данных
#data_all = pd.read_csv('news_lenta.csv')
#data_all.iloc[:100*1000,:].to_csv('part_news_lenta.csv', index=False)
#удалил целый файл news_lenta.csv, тк он оч тяжелый, оставил первые 100к строк в part_news_lenta.csv
data_all = pd.read_csv('part_news_lenta.csv')

In [5]:
def DropColumns(df, columns_list):
    '''Удаление столбцов в датафрейме'''
    return df.drop(columns_list, axis = 1)

def HandleMissedData(df, method='skip'):
    '''В tags есть данные с пропусками, применим 2 стратегии: удаление,
    замена на новое значение'''
    if method == 'delete':
        #пропуски составляют менее 5% от выборки, что является весьма малым и 
        #имеет смысл попробовать их просто удалить
        return df.dropna()
    else:
        return df.fillna(value='Пропуск')

def EncodeTarget(y):
    '''Для представения целевой переменной в численно виде'''
    lab_enc = LabelEncoder()
    return lab_enc.fit_transform(y)

def EncodeTags(train, test, validation, method='ohe'):
    '''Кодировка tags'''
    
    if method == 'ohe':
        one_hot_enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
        train = (one_hot_enc.fit_transform(train.values.reshape(-1,1)))
        test = (one_hot_enc.transform(test.values.reshape(-1,1)))
        validation = (one_hot_enc.transform(validation.values.reshape(-1,1)))
    #использовалось еще при применении деревьев
    elif method == 'target':
        targ_enc = TargetEncoder().fit(train.values, y_train)
        train = targ_enc.transform(train.values)
        test = targ_enc.transform(test.values)
        validation = targ_enc.transform(validation.values)
    return train, test, validation

def EncodeText(train, test, validation):
    '''Кодировка текстовых данных путем TF-IDF'''
    
    vectorizer = TfidfVectorizer(min_df=2, max_df=0.95,ngram_range=(1,2),max_features=10000)
    train = vectorizer.fit_transform(train)
    test = vectorizer.transform(test)
    validation = vectorizer.transform(validation)
    return train, test, validation

def RemainLettersNumsInLowerCase(data, columns_list):
    '''Оставить только буквы в нижнем регистре и цифры'''
    for column in columns_list:
        data[column] = data[column].apply(lambda x: re.sub('[\W]+', ' ', x.lower()))
        
def ThrowStopWords(series_column, stopwords_dict):
    '''Получение датафрейма с текстои и выбрасывание русских стоп-слов'''
    series = []
    for i, string in enumerate(series_column):
        series.append(' '.join(word for word in string.split() if stopwords_dict[word]==0))
    return series

def TrainTestValidationSplit(df):
    '''Деление выборки на train, test, validation'''
    
    #так как в выборке есть объекты встречающиеся 1-2 раза, произведем их добавление вручную так, чтобы
    #хотя бы 1 объект был в train, так же это полезно, когда для работы берется часть выборки
    #с целью сократить временные затраты, в таких случаях также наблюдаются редкие объекты
    
    small_topics = []
    val_counts = data['topic'].value_counts()
    for ix, counts in enumerate(val_counts):
        if counts <= 2:
            small_topics.append(val_counts.index[ix])
    
    indexes = {}
    all_ix = []
    for x in small_topics:
        l = []
        for ix in df[df['topic'] == x].index:
            l.append(ix)
            all_ix.append(ix)
        indexes[x] = l
        
    #удаление редких объектов из датасета, которые впоследствии будут добавлены вручную в train, test, validation
    df_modif = df.drop(all_ix, axis = 0)
    
    #для выделения данных в случае, если была склейка text и title или нет
    col_list = []
    if 'fulldiscr' in df_modif.columns:
        col_list.append('fulldiscr')
    else:
        col_list.append('text')
        col_list.append('title')
    col_list.append('tags')
    
    #деление на train, test, validation со стратификацией в соотношение 60% : 20% : 20%
    X_train, X_test, y_train, y_test = train_test_split(df_modif.loc[:, col_list], df_modif['topic'],
                                                        stratify=df_modif['topic'], test_size=0.2,
                                                        random_state=SEED)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train,
                                                      test_size=0.25, random_state=SEED)
    #добавление редких объектов
    for x, y_list in indexes.items():
        length_y = len(y_list)
        if length_y == 1:
            X_train = X_train.append(df.loc[y_list[0], col_list])
            y_train=y_train.append(pd.Series(df.loc[y_list[0], 'topic']))
        elif length_y == 2:
            X_train = X_train.append(df.loc[y_list[0], col_list])
            X_test = X_test.append(df.loc[y_list[1], col_list])
            y_train=y_train.append(pd.Series(df.loc[y_list[0], 'topic']))
            y_test=y_test.append(pd.Series(df.loc[y_list[1], 'topic']))
            
    return X_train, X_test, X_val, y_train, y_test, y_val

In [26]:
def TrainXgboost(X_train, X_test, y_train, y_test, n_classes, n_est=100, eta=0.05):
    '''Обучение XGBoost классификатора и выдача accuracy'''
    start = datetime.datetime.now()
    params = {}
    params['booster'] = 'gbtree'
    params['objective'] = 'multi:softmax'
    params['nthread'] = multiprocessing.cpu_count()
    params['num_class'] = n_classes
    params['seed'] = SEED
    #параметры получены на сетке на неполной выборке(иначе бы не дождался)
    params['n_estimators'] = n_est
    params['eta'] = eta
    
    X_train = scipy.sparse.csc_matrix(X_train)
    X_test = scipy.sparse.csc_matrix(X_test)
    
    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_test = xgb.DMatrix(X_test, label=y_test)

    clf = xgb.train(params, xgb_train)
    acc_score = accuracy_score(y_test, clf.predict(xgb_test))
    
    print('Time taken: {0}'.format(datetime.datetime.now() - start))
    return acc_score



In [7]:
def TrainLinearSVC(X_train, X_test, y_train, y_test, flag_balance=False):
    '''Обучение LinearSVC с балансировкой классов и без'''
    start = datetime.datetime.now()
    if flag_balance == True:
        clf_svc = LinearSVC(class_weight='balanced')
    else:
        clf_svc = LinearSVC()
    clf_svc.fit(X_train, y_train)
    print('Time taken: {0}'.format(datetime.datetime.now() - start))
    return clf_svc
    

In [8]:
def TrainNB(X_train, X_test, y_train, y_test, flag_balance=False):
    '''Обучение наивного байесовского классификатора с технологией OneVsRest'''
    start = datetime.datetime.now()
    clf_nb = MultinomialNB()
    clf = OneVsRestClassifier(clf_nb)
    clf.fit(X_train, y_train)
    print('Time taken: {0}'.format(datetime.datetime.now() - start))
    return clf

In [9]:
#Удалим сразу url, так как полезной информации, на мой взгляд, он не несет, за исключением даты,
#которая по большей части носит случайный характер
#но можно попробовать посчитать частоту встречаемости какой-то статьи среди всех статей за опр прмежуток
#data['url'] = data.loc[:,'url'].map(lambda x: datetime.datetime.strptime(re.search('\d+/\d+/\d+',x).group(), '%Y/%m/%d').date())
#data.rename({'url':'date'}, inplace=True, axis='columns')
data_all = DropColumns(data_all, 'url')

In [10]:
data_all.head(2)

,tags,text,title,topic
0,Общество,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир
1,Рынки,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика


In [11]:
#Для ускорения процесса возьмем часть выборки
data = data_all[:100*1000].copy()

In [12]:
#Обработаем пропущенные значения в данных
data = HandleMissedData(data)

In [13]:
#Закодируем topic в числа
data.loc[:,'topic'] = EncodeTarget(data.loc[:, 'topic'])

In [14]:
#Объединим 2 признака, так как, кажется, это дает выигрыш в качестве
data['fulldiscr'] = data['title'] + ' ' + data['text']


In [15]:
#Оставим в тексте только слова маленькими буквами и цифры 
RemainLettersNumsInLowerCase(data,['fulldiscr'])

In [16]:
#Выбросим русские стоп-слова
#data['fulldiscr'] = ThrowStopWords(data['fulldiscr'],stopwords_rus_dict)
#Попытка показывает небольшое ухудшение классификаторов (на 1-2%)

In [17]:
#Деление выборки на train, test, validation и проверка, что пропорции корректны
X_train, X_test, X_val, y_train, y_test, y_val = TrainTestValidationSplit(data)
print(X_train.shape[0]==y_train.shape[0],X_test.shape[0]==y_test.shape[0],X_val.shape[0]==y_val.shape[0])
print(X_train.shape[0]/data.shape[0],X_test.shape[0]/data.shape[0],X_val.shape[0]/data.shape[0])

True True True
0.6 0.2 0.2


In [18]:
#Остается произвести кодирование данных числами
##Начнем с tags
###one-hot encoding
train_tags, test_tags, val_tags = EncodeTags(X_train['tags'], X_test['tags'], X_val['tags'])

In [19]:
##Кодирование title и text
###TF-IDF кодирование
train_text, test_text, val_text = EncodeText(X_train['fulldiscr'],X_test['fulldiscr'],X_val['fulldiscr'])

In [20]:
#Объединение преобразованных массивов в один
X_train_modif = np.hstack([train_text.toarray(), train_tags])
X_test_modif = np.hstack([test_text.toarray(), test_tags])
X_val_modif = np.hstack([val_text.toarray(), val_tags])

###для scaled данных
#X_train_modif = np.hstack([train_text_sc, train_tags])
#X_test_modif = np.hstack([test_text_sc, test_tags])
#X_val_modif = np.hstack([val_text_sc, val_tags])

### try diff algos

In [27]:
acc = TrainXgboost(X_train_modif, X_test_modif, y_train, y_test, len(np.unique(y_train)))
print(acc)
#Time taken: 0:01:28.940291
#0.9028

Time taken: 0:06:51.688717
0.93255


In [70]:
#Обучим линейный SVM без балансировки классов
clf_SVC = TrainLinearSVC(X_train_modif, X_test_modif, y_train, y_test)
print(accuracy_score(y_test, clf_SVC.predict(X_test_modif)))

Time taken: 0:00:17.339599
0.9732


In [75]:
#Обучим линейный SVM с балансировкой классов
clf_SVC_bal = TrainLinearSVC(X_train_modif, X_test_modif, y_train, y_test, True)
print(accuracy_score(y_test, clf_SVC_bal.predict(X_test_modif)))

Time taken: 0:00:23.068698
0.97415


In [72]:
#Обучим байесовский классификатор
clf_nb = TrainNB(X_train_modif, X_test_modif, y_train, y_test)
print(accuracy_score(y_test, clf_nb.predict(X_test_modif)))

Time taken: 0:00:57.591083
0.94655


In [76]:
#Посмотрим на макро усредненные F1 меры у лучших классификаторов
ConfusionMatrix(actual_vector=list(y_test), predict_vector=list(clf_SVC.predict(X_test_modif))).F1_Macro, \
ConfusionMatrix(actual_vector=list(y_test), predict_vector=list(clf_SVC_bal.predict(X_test_modif))).F1_Macro

(0.959871343465076, 0.9610108931956591)

In [78]:
#Проверим показатели accuracy и F1_macro на валидационной выборке у лучшего классификатора
print(accuracy_score(y_val, clf_SVC.predict(X_val_modif)))
print(ConfusionMatrix(actual_vector=list(y_val), predict_vector=list(clf_SVC.predict(X_val_modif))).F1_Macro)

0.97415
0.912097349104367
